In [1]:
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertModel
from tqdm import tqdm
import numpy as np

d:\SoftwareData\Anaconda3\envs\topic\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### 加载文件，这里最好使用原始文本，不要使用分词过后的文本

In [2]:
with open('../data/文本.txt', 'r', encoding='utf-8') as file:#这个文本是切词之前的文本
  sentences = file.readlines()
  # sentences = file.readlines()[:100] # ⚠️测试的时候可以只看前100条数据
print('文本条数: ', len(sentences))
print('预览第一条: ', sentences[0])

文本条数:  1000
预览第一条:  【#文旅文创看洛阳# 】2023#河南省文旅文创发展大会# 本次大会安排了项目签约，主要有两方面内容。一是文旅产业项目签约。截至目前，共梳理41个重点文旅项目，投资总额525.6亿元；遴选21个重大项目进行现场签约，投资总额365.8亿元。这些项目既包括文物数字化开发、文化创意园区建设等文化类项目，也涵盖了旅游度假区建设、旅游酒店民宿打造等旅游类项目，既有旅游景区开发、文商旅综合体建设等传统业态项目，也有元宇宙基地、沉浸式演艺等新业态项目，充分体现了我省文化旅游发展的特点和趋势。二是引客入豫项目签约。主要是我省文旅部门、文旅企业与头部旅行商、知名OTA平台、重点客源地文旅部门等签订引客入豫协议等，持续拓展省外客源市场。



In [3]:
# 加载预训练模型和tokenizer
# 模型名字直接写入bert-base-chinese这个简化模型名就可以了，https://huggingface.co/google-bert/bert-base-chinese
# 如果无法用梯子的话，可以本地下载：huggingface-cli download --resume-download bert-base-chinese
model_name = "bert-base-chinese"
# model_name = "hfl/chinese-bert-wwm"

# 也可以试试使用哈工大的模型，model_name = "hfl/chinese-bert-wwm"
# 注意提前需要下载huggingface-cli download --resume-download hfl/chinese-bert-wwm

# 加载模型
# 会从huggingface中下载模型
# 源码：class PreTrainedModel(nn.Module....)
# 所以，这里创建的既是PreTrainedModel类的实例，也是torch.nn.Module的实例
# 对于警告Some weights of the model checkpoint，对与我们的任务，可以不用在意
# 相关讨论：https://blog.csdn.net/PolarisRisingWar/article/details/123974645   https://huggingface.co/google-bert/bert-base-uncased/discussions/4

# 从镜像站下载
# conda activate test
# pip install -U huggingface_hub
# $env:HF_ENDPOINT = "https://hf-mirror.com"
# huggingface-cli download --resume-download bert-base-chinese   或者使用哈工大模型   huggingface-cli download --resume-download hfl/chinese-bert-wwm
model = BertModel.from_pretrained(model_name)#sentence transformer可能是需要填入具体地址的

# 加载tokenizer
# 使用Tokenizer，就是为了将输入的句子加工为bert模型可以处理的格式
tokenizer = BertTokenizer.from_pretrained(model_name)

d:\SoftwareData\Anaconda3\envs\topic\lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exact

In [4]:
# 将模型放置在GPU上
# torch.cuda.is_available()，检测cuda是否可用
# torch.device()设置张量运算在哪个设备上进行
# device = torch.device("cpu")，表示在CPU进行
# device = torch.device("cuda")，表示在GPU进行
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 把模型放到cpu或gpu
model.to(device)
# 将模型设置为评估模式，https://blog.csdn.net/weixin_45275599/article/details/131524189
model.eval()

# 切分数据
batch_size = 16  # 批大小
data_loader = DataLoader(sentences, batch_size=batch_size)
for batch in data_loader:
    print(len(batch), batch)

16 ['【#文旅文创看洛阳# 】2023#河南省文旅文创发展大会# 本次大会安排了项目签约，主要有两方面内容。一是文旅产业项目签约。截至目前，共梳理41个重点文旅项目，投资总额525.6亿元；遴选21个重大项目进行现场签约，投资总额365.8亿元。这些项目既包括文物数字化开发、文化创意园区建设等文化类项目，也涵盖了旅游度假区建设、旅游酒店民宿打造等旅游类项目，既有旅游景区开发、文商旅综合体建设等传统业态项目，也有元宇宙基地、沉浸式演艺等新业态项目，充分体现了我省文化旅游发展的特点和趋势。二是引客入豫项目签约。主要是我省文旅部门、文旅企业与头部旅行商、知名OTA平台、重点客源地文旅部门等签订引客入豫协议等，持续拓展省外客源市场。\n', '#五一# #新书速递# “班门”系列上新啦！《班门·广场》聚焦四个大洲，十余个国家、五十余处广场。跨越学科，关注日常，从建筑、历史、文化等角度重新理解作为公共空间的广场，五一出游，感受城市中的流动生命。“只有来到城市的主要广场，才算真正抵达城市”打开《班门·广场》，开始一场穿越时空的盛大卧游：感受古老广场被岁月磨砺出的各异气质，领略清华大学大礼堂广场自然山水与人工建筑的融合，追溯飞迸的罗马广场喷泉背后贵族权力与公共利益的博弈，见证中国古代“广场”之形状……我们也将广场的线索延伸至当下乃至未来，领略洛阳隋唐西市广场的设计理念、慕尼黑大小广场传统与现代交织的布局方式和艺术风格、以“打破建筑传统”为目标的扎哈·哈迪德在韩国东大门广场设计中体现的超前思考与表达……试图从中窥测未来广场的发展趋势。设计者、旅行者、作者，图上、路上、纸上。《班门·广场》以摄影、绘画、写作来丈量广场的源流与变迁，捕捉广场特有的艺术底蕴与勃发的生命力。\n', '#行走河南阅见美好#“一镜到底”、“和白居易一起喝茶听曲”、“白居易谈自己喝酒后也会被老婆大人管教”……种种奇幻般的镜头于昨日在洛阳白园发生了。网友们纷纷表示听白居易老先生一席话胜读十年河南史。原来，二十六日下午，洛阳举办了“行走河南阅见美好”带着书本去旅行系列活动，第一期的“唐‘潮’寻梦路”慢直播微综艺活动在白园举行，白居易老先生穿越时空应邀而来。在鸟语花香的园林里，白居易老先生与现代著名作家王小朋先生时而谈诗论赋、喝茶听曲，时而谈古论今、传道受业解惑，时而讨论人生、感叹生活点滴。两位大佬引经据典，谈话

In [6]:
# ---- 文本转向量 ----
# 生成的向量存放在这里
cls_embeddings = []

# 使用tqdm显示处理进度
# tqdm b站教程：https://www.bilibili.com/video/BV1ZG411M7Ge/?spm_id_from=333.337.search-card.all.click&vd_source=eace37b0970f8d3d597d32f39dec89d8
for batch_sentences in tqdm(data_loader):
    # tokenizer官方文档：https://huggingface.co/docs/transformers/main_classes/tokenizer#transformers.PreTrainedTokenizer.__call__
    # truncation=True，对输入句子进行截断，这里确保最大长度不超过512个字
    # max_length：不设置的话，默认会截断到该模型可接受的最大长度
    # padding=True 或 padding='longest': 将所有句子填充到批次中最长句子的长度
    # padding="max_length": 将所有句子填充到由 max_length 参数指定的长度
    inputs = tokenizer(batch_sentences, padding=True, truncation=True, return_tensors="pt", max_length=512)
    # print(123, inputs.input_ids[0], tokenizer.decode(inputs.input_ids[0]))#展示文本经过分词过后的形式是什么样子的
    
    # 把编码好的数据，也放在device上，It is necessary to have both the model, and the data on the same device, either CPU or GPU
    # https://huggingface.co/docs/transformers/v4.39.2/en/main_classes/tokenizer#transformers.BatchEncoding.to
    # https://stackoverflow.com/questions/63061779/pytorch-when-do-i-need-to-use-todevice-on-a-model-or-tensor
    inputs.to(device)

    # 设置不要计算梯度
    # 一般来说，如果我们只是用模型进行“预测”，而不涉及对模型进行更新时，就不需要计算梯度，以此来节约内存，增加运算效率
    # with上下文中，对model的调用将遵循torch.no_grad()，即不会计算梯度
    with torch.no_grad():
        outputs = model(**inputs)

    # 把这一批词向量存入cls_embeddings容器中
    # tensor.cpu() 将张量移动到 CPU
    # tensor.numpy() 将 CPU 上的张量转换为 NumPy 数组
    cls_embeddings.append(outputs.last_hidden_state[:, 0].cpu().numpy()) # 只取CLS对应的向量

    # print('pt格式', type(outputs.last_hidden_state[:, 0].shape), outputs.last_hidden_state[:, 0].shape)
    print('numpy格式', type(outputs.last_hidden_state[:, 0].cpu().numpy()), outputs.last_hidden_state[:, 0].cpu().numpy().shape)

# 合并句子向量
print('batch个数：', len(cls_embeddings))
cls_embeddings_np = np.vstack(cls_embeddings)
print('最终生成的词向量', type(cls_embeddings_np), cls_embeddings_np.shape)

# ---- 保存词嵌入向量 ----
# 保存句子向量到npy文件
# 官方文档：https://numpy.org/doc/stable/reference/generated/numpy.save.html


#11111111111111111111111111111111保存成文件，然后拉到data中去111111111111111111111
output_file = "emb.npy"
np.save(output_file, cls_embeddings_np)
print("词向量存储于: ", output_file)

embeddings = np.load(output_file)
print("加载回来，验证一下：", type(embeddings), embeddings.shape)

  2%|▏         | 1/63 [00:00<00:24,  2.48it/s]

numpy格式 <class 'numpy.ndarray'> (16, 768)


  3%|▎         | 2/63 [00:00<00:21,  2.84it/s]

numpy格式 <class 'numpy.ndarray'> (16, 768)


  5%|▍         | 3/63 [00:01<00:21,  2.77it/s]

...

numpy格式 <class 'numpy.ndarray'> (16, 768)


 97%|█████████▋| 61/63 [00:21<00:00,  2.88it/s]

numpy格式 <class 'numpy.ndarray'> (16, 768)


100%|██████████| 63/63 [00:22<00:00,  2.86it/s]

numpy格式 <class 'numpy.ndarray'> (16, 768)
numpy格式 <class 'numpy.ndarray'> (8, 768)
batch个数： 63
最终生成的词向量 <class 'numpy.ndarray'> (1000, 768)
词向量存储于:  emb.npy
加载回来，验证一下： <class 'numpy.ndarray'> (1000, 768)
